In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('pred_lstm18.csv')
dataset_2019 = pd.read_csv('pred_lstm19.csv')
dist = pd.read_csv('dataset/Distance_Matrix.csv')
dist.drop(columns="Unnamed: 0", inplace=True)

In [3]:
arr = [i[0] for i in dataset.values]
arr1 = [i[0] for i in dataset_2019.values]
arr2 = [min(arr[i], arr1[i]) for i in range(len(arr))]

print(sum(arr1), sum(arr), sum(arr2))

336415.76609212795 304039.6590211247 264896.4189262302


In [4]:
sad = [list(i) for i in list(dist.values)]

In [5]:
min(arr)

0.2232165566111111

In [14]:
groups = [[], [], []]

def track(ar):
    one = 0
    for i in ar:
        one += arr[i]
    return one

def track2(ar):
    one = 0
    for i in ar:
        one += arr1[i]
    return one

i=0
keep = set()
for index, val in enumerate(dist.values):
    if i <= 2:
        if index not in keep:
            print(i)
            while track(groups[i]) < 101500:
                minIndex = -1
                minVal = float('inf')
                for x,y in enumerate(val):
                    if x not in keep:
                        if dist.values[x][index] < minVal:
                            minVal = dist.values[x, index]
                            minIndex = x
                if minIndex == -1:
                    break
                keep.add(minIndex)
                groups[i].append(minIndex)
            i+= 1

0
1
2


In [15]:
for i in groups:
    print(track(i))

101625.42383759763
101594.08563439618
100820.14954913106


In [19]:
dog = groups[0][:100]

from pulp import *
import random as rnd

sites = [f"{i}" for i in dog]
depots = [f"{i}" for i in dog]

depot_me = [f"{i}" for i in dog]

supply = {f"{i}":arr[i] for i in dog}
# supply_2019 = {f"{i}":arr1[i] for i in dog}

refineries = [f"{i}" for i in dog]

demand = {f"{i}":19999.99 for i in dog}
demand_ref = {f"{i}":99999.99 for i in dog}

sad = [list(i) for i in list(dist.values)]
distance = sad
distance = makeDict([sites, depots], distance, 0)

# distance_ref = sad
# distance_ref = makeDict([depot_me, refineries], distance_ref, 0)

prob = LpProblem("Depot_Problem", LpMinimize)

Routes = [(w, b) for w in sites for b in depots]

depot_var = LpVariable.dicts("Supply_to_depots",depots, 0, 1, LpInteger)

vars = LpVariable.dicts("Route", (sites, depots), 0, None, LpInteger)

# Ref
# ref_vars = LpVariable.dicts("Route_ref", (depot_me, refineries), 0, None, LpInteger)
# ref_var = LpVariable.dicts("Supply_to_refineries",refineries, 0, 1, LpInteger)
# Routes_ref = [(w, b) for w in depot_me for b in refineries]

main_obj = (lpSum(vars[w][b] * distance[w][b] for (w, b) in Routes)*0.001)
# ref_constraint = (lpSum(ref_vars[b][k] * distance[b][k] for (b, k) in Routes_ref)*0.001)

prob += main_obj

for w in sites:
    prob += (
        lpSum(vars[w][b] for b in depots) <= supply[w],
        f"Sites must not give more than they produce{w}"
    )

#Refineries
# for b in depot_me:
#     if depot_var[b] >= 1:
#         prob += (
#             lpSum(ref_vars[b][k] for k in refineries) <= (lpSum(vars[w][b] for w in sites)),
#             f"Depot must not give more than they produce{b}"
#         )

for b in depots:
    if depot_var[b] >= 1:
        prob += (
            lpSum([vars[w][b] for w in sites]) <= demand[b]*depot_var[b],
            f"Sum of products into depot should be less than its capacity {b}"
        )

#Refineries
# for b in refineries:
#     if ref_var[b] >= 1:
#         prob += (
#             lpSum([ref_vars[j][b] for j in depots]) <= demand_ref[b]*ref_var[b],
#             f"Sum of products into refineries should be less than its capacity {b}"
#         )

prob += lpSum(depot_var[b] for b in depots) <= max(1, min(25,(0.8*sum(supply.values())//20000)+1))
# prob += lpSum(depot_var[b] for b in depots) <= max(1, min(25,(0.8*sum(supply_2019.values())//20000)+1))
# prob += lpSum(ref_var[b] for b in refineries) <= max(1, min(5,(0.8*sum(supply.values())//100000)+1))
# prob += lpSum(ref_var[b] for b in refineries) <= max(1, min(5,(0.8*sum(supply_2019.values())//100000)+1))

huf = ()
for w in sites:
    for b in depots:
        if depot_var[b] >= 1:
            huf += (
            lpSum(vars[w][b] )
            )
        

# prob += lpSum(huf) >= 99900
prob += lpSum(huf) >= sum(supply.values())*0.9
# prob += lpSum(huf) >= sum(supply_2019.values())*0.8

# Contraint 8: Amount entering into depot equals amount exiting depot
# for b in depots:
#     if depot_var[b] >= 1:
#         prob += (
#             lpSum(vars[w][b] for w in sites) <= lpSum(ref_vars[b][k] for k in refineries if ref_var[k] >= 1)
#         )
    
# Refineries
# hufe = ()
# for w in depots:
#     for b in refineries:
#         if ref_var[b] >= 1:
#             hufe += (
#             lpSum(ref_vars[w][b] )
#             )

# prob += lpSum(hufe) >= 99999.5
# prob += (lpSum(hufe) >= sum(supply.values())*0.8)
# prob += lpSum(hufe) >= sum(supply_2019.values())*0.8

solver_options = {'msg': True, 'timeLimit': 8000}  # Set solver-specific options
prob.solve(CPLEX_CMD(**solver_options))

# for i in vars:
#     for each in vars[i]:
#         if vars[i][each].varValue > 0:
#             print(f"TP {vars[i][each]} ---  {vars[i][each].varValue}")
# print("\nSupply ref ")
# for i in ref_vars:
#     for each in ref_vars[i]:
#         if ref_vars[i][each].varValue > 0:
#             print(f"TP {ref_vars[i][each]} ---  {ref_vars[i][each].varValue}")
            
print("Supplied Depots:")
for b in depots:
    if depot_var[b].varValue == 1:
        print(b)

# print("\nSupplied Refineries:")
# for b in refineries:
#     if ref_var[b].varValue == 1:
#         print(b)

print(f"\n{sum(supply.values())}")
print((sum(supply.values())//20000)+1)
print(sum([(lpSum(vars[w][b].varValue for w in sites)) for b in depots if depot_var[b].varValue >= 1]))
# print(sum([(lpSum(ref_vars[w][b].varValue for w in depots)) for b in refineries if ref_var[b].varValue >= 1]))
print([depot_var[i].varValue for i in depot_var if depot_var[i] == 1])
# print(supply.values())

Supplied Depots:
96

8308.911726363094
1.0
7479.0
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [16]:
([(lpSum(vars[w][b].varValue for w in sites)) for b in depots if depot_var[b].varValue >= 1])

[19904.0, 19999.0, 19999.0, 19999.0, 19999.0]

In [ ]:
 1024.77 sec.

In [ ]:
dist.values[52][0]

In [27]:
give = []
receive = []
amount = []
for i in vars:
    for each in vars[i]:
        if vars[i][each].varValue > 0:
            give.append(int(i))
            receive.append(int(each))
            amount.append(vars[i][each].varValue)
            print(f"{[int(i), int(each)]} ---  {vars[i][each].varValue}")

[49, 385] ---  108.0
[5, 385] ---  31.0
[118, 385] ---  20.0
[121, 385] ---  121.0
[122, 385] ---  177.0
[75, 385] ---  211.0
[120, 385] ---  104.0
[169, 385] ---  13.0
[91, 385] ---  1.0
[76, 385] ---  174.0
[170, 607] ---  56.0
[54, 607] ---  190.0
[208, 607] ---  74.0
[124, 607] ---  146.0
[209, 385] ---  74.0
[125, 385] ---  143.0
[210, 385] ---  85.0
[149, 385] ---  177.0
[148, 385] ---  98.0
[172, 385] ---  95.0
[126, 385] ---  175.0
[249, 385] ---  111.0
[147, 385] ---  169.0
[250, 607] ---  73.0
[39, 607] ---  82.0
[150, 607] ---  183.0
[175, 607] ---  150.0
[77, 607] ---  100.0
[55, 607] ---  143.0
[8, 607] ---  52.0
[174, 607] ---  157.0
[40, 385] ---  54.0
[78, 385] ---  165.0
[289, 385] ---  56.0
[23, 385] ---  172.0
[252, 385] ---  245.0
[151, 385] ---  149.0
[57, 385] ---  127.0
[288, 385] ---  12.0
[152, 385] ---  100.0
[102, 385] ---  151.0
[56, 385] ---  147.0
[24, 385] ---  171.0
[37, 607] ---  172.0
[103, 607] ---  117.0
[247, 607] ---  5.0
[177, 607] ---  134.0
[214

[755, 929] ---  168.0
[1167, 929] ---  186.0
[800, 929] ---  385.0
[987, 929] ---  213.0
[1049, 929] ---  232.0
[1094, 929] ---  451.0
[667, 858] ---  3.0
[1037, 858] ---  261.0
[1165, 858] ---  99.0
[1044, 858] ---  109.0
[787, 858] ---  6.0
[1093, 858] ---  464.0
[1109, 858] ---  54.0
[1092, 858] ---  418.0
[734, 858] ---  200.0
[398, 858] ---  77.0
[1095, 858] ---  173.0
[927, 607] ---  190.0
[988, 607] ---  223.0
[588, 607] ---  17.0
[468, 607] ---  30.0
[1005, 607] ---  221.0
[1161, 499] ---  275.0
[687, 499] ---  137.0
[1035, 499] ---  231.0
[853, 499] ---  1.0
[1159, 499] ---  299.0
[622, 499] ---  249.0
[529, 499] ---  33.0
[450, 499] ---  135.0
[354, 499] ---  44.0
[563, 499] ---  166.0
[1036, 499] ---  321.0
[1006, 499] ---  138.0
[1058, 499] ---  134.0
[756, 499] ---  114.0
[1098, 499] ---  238.0
[1221, 929] ---  26.0
[1223, 929] ---  170.0
[1150, 929] ---  175.0
[1034, 929] ---  34.0
[1162, 929] ---  141.0
[1148, 929] ---  451.0
[649, 929] ---  13.0
[1107, 929] ---  228.0
[

In [36]:
bro = {}

bro['sites'] = give
bro['depots'] = receive
bro['value'] = amount

df = pd.DataFrame(bro)
df.to_csv("group_0.csv")
df.head()

,sites,depots,value
0,49,385,108.0
1,5,385,31.0
2,118,385,20.0
3,121,385,121.0
4,122,385,177.0


In [19]:
for i in vars:
    for each in vars[i]:
        if vars[i][each].varValue > 0:
            print(i)

49
5
118
121
122
75
120
169
91
76
170
54
208
124
209
125
210
149
148
172
126
249
147
250
39
150
175
77
55
8
174
40
78
289
23
252
151
57
288
152
102
56
24
37
103
247
177
214
25
211
127
22
290
293
253
287
291
213
246
21
7
215
58
128
104
26
178
256
81
335
80
328
334
286
130
42
11
153
296
59
257
336
131
179
154
105
217
43
332
295
106
379
337
331
60
377
132
28
258
155
180
338
156
157
297
378
259
27
216
376
339
133
427
424
158
182
382
83
374
285
299
260
340
298
183
184
61
108
244
220
423
480
380
429
84
425
243
486
221
134
428
159
430
431
383
433
432
261
487
373
341
85
160
481
222
434
479
262
546
135
109
301
242
485
202
224
422
489
325
372
488
185
342
136
63
263
264
541
86
421
302
490
478
343
241
161
435
265
544
385
606
477
371
201
605
137
303
484
110
540
607
304
547
549
111
266
344
436
604
548
200
240
386
601
305
539
323
324
600
420
187
483
345
491
538
672
306
609
138
437
346
199
543
370
267
188
239
387
674
162
348
610
322
198
741
671
307
369
476
675
599
419
673
112
388
347
492
608
225
598
7